In [ ]:
import onnx
import numpy as np
from onnx import numpy_helper
from onnxconverter_common import float16
import copy

def sparsify_onnx_weights(model, k):
    """
    AID_Pruning_Exercise.ipynb의 'sparsify_by_weights' 로직을 ONNX에 적용.
    가중치의 절대값 크기가 작은 하위 k(비율)만큼을 0으로 만듭니다.
    """
    print(f"Applying Weight Pruning (Sparsity: {k})...")
    
    # 모델의 모든 초기화 값(Weights, Biases 등)을 순회
    # ONNX에서 가중치는 graph.initializer에 저장됩니다.
    total_weights = len(model.graph.initializer)
    
    for idx, tensor in enumerate(model.graph.initializer):
        # ONNX Tensor를 Numpy 배열로 변환
        weight_array = numpy_helper.to_array(tensor)
        
        # 1. 안전 장치: 1차원 배열(Bias, Scale 등)은 건너뛰고, 
        #    다차원 배열(Conv/Linear Weights)만 프루닝하는 것이 일반적입니다.
        if weight_array.ndim < 2:
            continue
            
        # 2. 노트북의 프루닝 로직 적용
        # (flatten -> 절대값 기준 정렬 -> 하위 k% 인덱스 추출 -> 0으로 설정)
        layer_copy = weight_array.flatten()
        
        # 절대값이 작은 순서대로 정렬하여 인덱스 추출
        indices = np.abs(layer_copy).argsort()
        
        # 하위 k%에 해당하는 개수 계산
        n_prune = int(len(indices) * k)
        
        # 해당 인덱스들을 0으로 설정
        indices_to_zero = indices[:n_prune]
        layer_copy[indices_to_zero] = 0
        
        # 3. 변경된 Numpy 배열을 다시 원래 형태(Shape)로 복구
        pruned_weight = layer_copy.reshape(weight_array.shape)
        
        # 4. ONNX Tensor 업데이트
        # 기존 데이터를 대체하기 위해 새로운 Tensor 생성
        new_tensor = numpy_helper.from_array(pruned_weight, tensor.name)
        
        # 리스트 내의 해당 텐서를 교체 (직접 할당 불가하므로 copy 및 replace 방식 사용)
        model.graph.initializer.remove(tensor)
        model.graph.initializer.append(new_tensor)

    print("Pruning Complete.")
    return model

def apply_fp16(model):
    """
    ONNX 모델을 FP16(Half Precision)으로 변환합니다.
    """
    print("Converting to FP16...")
    fp16_model = float16.convert_float_to_float16(model)
    print("FP16 Conversion Complete.")
    return fp16_model

# --- 메인 실행 코드 ---
if __name__ == "__main__":
    input_model_path = "best6.onnx"
    output_model_path = "best6_pruned_fp16.onnx"
    sparsity_level = 0.2  # 예제 노트북과 같이 20% 희소성 적용 (필요시 조절)

    # 1. 모델 로드
    print(f"Loading {input_model_path}...")
    model = onnx.load(input_model_path)

    # 2. 프루닝 적용 (노트북 로직)
    model = sparsify_onnx_weights(model, k=sparsity_level)

    # 3. FP16 변환 적용
    model = apply_fp16(model)

    # 4. 저장
    onnx.save(model, output_model_path)
    print(f"Saved processed model to {output_model_path}")

Loading model : ./best.pt
Applying Pruning : (Ratio : 25.0%)
64 layers applied!
Applying Quantization and exporting onnx
Ultralytics 8.3.235  Python-3.12.10 torch-2.10.0a0+rocm7.10.0a20251106 CUDA:1 (AMD Radeon RX 7800 XT, 16368MiB)
Model summary (fused): 72 layers, 4,765,733 parameters, 0 gradients

PyTorch: starting from 'best.pt' with input shape (1, 3, 128, 128) BCHW and output shape(s) (1, 4807, 336) (18.5 MB)

ONNX: starting export with onnx 1.19.1 opset 11...
ONNX: slimming with onnxslim 0.1.78...
ONNX: export success  7.1s, saved as 'best.onnx' (9.2 MB)

Export complete (10.7s)
Results saved to C:\Users\garli\Downloads\yolo
Predict:         yolo predict task=detect model=best.onnx imgsz=128 half 
Validate:        yolo val task=detect model=best.onnx imgsz=128 data=data.yaml half 
Visualize:       https://netron.app
------------------------------
Exportation complete : best.onnx
